https://stackoverflow.com/questions/63201036/add-additional-layers-to-the-huggingface-transformers

https://stackoverflow.com/questions/64156202/add-dense-layer-on-top-of-huggingface-bert-model

https://stackoverflow.com/questions/63201036/add-additional-layers-to-the-huggingface-transformers

https://stackoverflow.com/questions/51093691/custom-activation-function-keras-applying-different-activation-to-different-lay

In [1]:
# Comprobamos si está tensorflow-gpu==2.3.0
!pip freeze

absl-py==0.12.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.1.0
appdirs==1.4.4
argon2-cffi==20.1.0
arviz==0.11.2
astor==0.8.1
astropy==4.2.1
astunparse==1.6.3
async-generator==1.10
atari-py==0.2.6
atomicwrites==1.4.0
attrs==21.2.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==3.3.0
blis==0.4.1
bokeh==2.3.2
Bottleneck==1.3.2
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.6
cachetools==4.2.2
catalogue==1.0.0
certifi==2020.12.5
cffi==1.14.5
cftime==1.4.1
chainer==7.4.0
chardet==3.0.4
click==8.0.0
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==2.0.6
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.3.2
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cupy-cuda101==7.4.0
cvxopt==1.2.6
cvxpy==1.0.31
cycler==0.10.0
cymem==2.0.5
Cython==0.29.23
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill==0.3.3
distributed==1.25.3
dlib==19.18.0
d

In [2]:
# Esta es la tarjeta grafica
!nvidia-smi

Tue May 25 10:31:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# instalar librerías. Esta casilla es últil por ejemplo si se ejecuta el cuaderno en Google Colab
# Note que existen otras dependencias como tensorflow, etc. que en este caso se encontrarían ya instaladas

%%capture
# Libreríía transformers
try:
    import tranformers
    print("module 'tranformers' is installed")
except ModuleNotFoundError:
    print("module 'transformers' is being installed")
    !pip install transformers==4.2.1
# Por si se quiere usar la gpu verificamos si tenemos transformers-gpu
import sys
if "tensorflow-gpu" in sys.modules:
    print("tensorflow-gpu already in sys.modules")
else: 
  !pip install tensorflow-gpu==2.3.0

In [4]:
!pip install tensorflow-gpu==2.3.0

In [5]:
import pandas as pd
import tensorflow as tf
from transformers import DistilBertConfig, TFDistilBertModel, DistilBertTokenizer
from sklearn import preprocessing
from sklearn.metrics import f1_score
# Para trabajar teniendo encuenta el desbalanceo
from sklearn.utils import class_weight
import os
import tqdm
import numpy as np
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split

In [6]:
train_dataframe = pd.read_csv("https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/haha_2021_train.csv", sep=',')
# According to https://www.fing.edu.uy/inco/grupos/pln/haha/
# we have to assume it is a joke so we can filter by humorous tweets
train_dataframe = train_dataframe.query('is_humor == 1')
train_dataframe.head(6)

,id,text,is_humor,votes_no,votes_1,votes_2,votes_3,votes_4,votes_5,humor_rating,humor_mechanism,humor_target
0,tweet1,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,1,1,2,2,0,0,0,1.50,NaN,NaN
1,tweet2,"—Vamos Luke desenfunda tu sable, demuestra tu ...",1,1,3,0,1,0,0,1.50,NaN,NaN
2,tweet3,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",1,0,2,1,0,1,1,2.60,NaN,NaN
6,tweet7,"—Buenas don Pepe, ¿me vende un litro de leche?...",1,1,1,1,1,1,0,2.50,NaN,NaN
12,tweet13,#20CosasQueHacerAntesDeMorir: Enseñarles la di...,1,0,1,1,2,1,0,2.60,reference,NaN
13,tweet14,Leí que la falta de sexo trae consigo una nota...,1,1,1,0,1,1,1,3.25,NaN,NaN


In [7]:
# Creo un conjunto de validación con data splitting
train, val = train_test_split(train_dataframe, test_size=.15)

In [8]:
x_train = train['text']
y_train = train['humor_rating']
x_val = val['text']
y_val = val['humor_rating']

In [9]:
cfg = {}
cfg["framework"] = "tf"
cfg["max_length"] = 256 # 380 caracteros maximo por tweet (Mirar maximo de longitud)
cfg["transformer_model_name"] = "dccuchile/bert-base-spanish-wwm-cased" # Este es el modelo Bert para Spanish, con mayusculas
cfg["num_labels"] = 1

In [10]:
# dim dimension del pooling layer de los outputs del encoder en la salida de la ultima capa
# https://huggingface.co/transformers/model_doc/distilbert.html
# https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased
# dropout es el dropout de las denses de las capas de encoders
# seq_classif_dropout es el dropout de la ultima densa ajena a Bert
# Reducimos la complejidad del problema, solo tenemos 25000 tweets

# cargar el tokenizador, disponible en Transformers
cfg['tokenizer'] = DistilBertTokenizer.from_pretrained(cfg['transformer_model_name'] )

# instanciar y entrenar LabelBinarizer
#cfg['label_binarizer'] = preprocessing.LabelBinarizer() # guardar para su posterior uso al decodificar predicciones

config_bert = DistilBertConfig(attention_dropout=0.6, dropout=0.6,
                               n_heads=2, dim=32, max_position_embeddings=cfg["max_length"],
                               n_layers=2, hidden_dim=64, vocab_size=cfg['tokenizer'].vocab_size, output_hidden_states=True)
# model = TFDistilBertModel.from_pretrained(cfg["transformer_model_name"], config=config_bert)

In [11]:
cfg['tokenizer'].vocab_size

31002

In [12]:
# ML Keras
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, BatchNormalization, LeakyReLU, ReLU, Activation, Dropout, Conv1D, Reshape, Flatten, MaxPooling1D, Lambda, Concatenate
from tensorflow.keras.activations import softmax, tanh, sigmoid, relu
from keras.utils import np_utils
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model


In [13]:
# Permite obtener las stop_words de los idiomas soportados
!pip install stop_words

In [14]:
from stop_words import get_stop_words # Se añade librería para obtener las stop_word de cualquier idioma
!wget "https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/more_stop_words.txt"

--2021-05-25 10:32:03--  https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/more_stop_words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10258 (10K) [application/octet-stream]
Saving to: ‘more_stop_words.txt.2’

more_stop_words.txt 100%[===================>]  10.02K  --.-KB/s    in 0s      

2021-05-25 10:32:03 (106 MB/s) - ‘more_stop_words.txt.2’ saved [10258/10258]



In [15]:
# Se obtienen las stop_words en español
import pickle
with open("more_stop_words.txt", "rb") as f:
  list_test = pickle.load(f)

stop_words = get_stop_words("spanish")
more_stop_words = stop_words + list_test
more_stop_words.sort()

In [16]:
from nltk.stem.snowball import SpanishStemmer
def spanish_stemmer(sentence):
    stemmer = SpanishStemmer()
    analyzer = CountVectorizer(binary=False, analyzer='word', stop_words=more_stop_words,
                               ngram_range=(1, 1)).build_analyzer()
    return (stemmer.stem(word) for word in analyzer(sentence))

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
c_vec = CountVectorizer(analyzer=spanish_stemmer, stop_words=more_stop_words)
# tf_idf = TfidfTransformer(smooth_idf=True, use_idf=True)
c_vec_mat = c_vec.fit_transform(x_train)
# tf_idf_mat = tf_idf.fit_transform(c_vec_mat)
c_vec_mat.shape

(7865, 9185)

In [18]:
df_c_vec = pd.DataFrame(c_vec_mat.toarray(), columns=c_vec.get_feature_names())
print(df_c_vec)

      00h  0_0  12456x  1245x⅜  138  ...  فيس  ماcualqui  مشغووقف  我不明白這我不明白這  ｇｉｍｉo
0       0    0       0       0    0  ...    0          0        0           0      0
1       0    0       0       0    0  ...    0          0        0           0      0
2       0    0       0       0    0  ...    0          0        0           0      0
3       0    0       0       0    0  ...    0          0        0           0      0
4       0    0       0       0    0  ...    0          0        0           0      0
...   ...  ...     ...     ...  ...  ...  ...        ...      ...         ...    ...
7860    0    0       0       0    0  ...    0          0        0           0      0
7861    0    0       0       0    0  ...    0          0        0           0      0
7862    0    0       0       0    0  ...    0          0        0           0      0
7863    0    0       0       0    0  ...    0          0        0           0      0
7864    0    0       0       0    0  ...    0          0        0

In [19]:
class CustomDistilBERTModel():
    def __init__(self, cfg, config_bert):

          # Configuraciones
          self.cgf = cfg
          self.config_bert = config_bert
          
          # Instanciamos DistilBertModel
          self.DistilBert = TFDistilBertModel.from_pretrained(self.cgf["transformer_model_name"],
                                                      config=self.config_bert)
          self.DistilBert_model = None

    def custom_activation(self, sigma):
      # https://stackoverflow.com/questions/51093691/custom-activation-function-keras-applying-different-activation-to-different-lay
      return 4*sigma+1

    """def custom_activation_shape(self, input_shape):
        # Ensure there is rank 4 tensor
        assert len(input_shape) == 4
        # Ensure the last input component has 5 dimensions
        assert input_shape[3] == 5

        return input_shape  # Shape is unchanged"""

    def get_model_inputs(self, cfg, data):
        encodings = cfg['tokenizer'](data, truncation=True, padding='max_length', max_length=cfg['max_length'], return_tensors=cfg['framework'])
        inputs = {'input_ids': encodings['input_ids'],
                'attention_mask': encodings['attention_mask']
                }
        return inputs

    def DistilBertNN(self):#, ids, mask):
          # https://stackoverflow.com/questions/64156202/add-dense-layer-on-top-of-huggingface-bert-model
          # https://stackoverflow.com/questions/63201036/add-additional-layers-to-the-huggingface-transformers
          input_ids = Input(shape=(256, ),dtype='int32')
          attention_mask = Input(shape=(256, ), dtype='int32')
          word_context = Input(shape=(c_vec_mat.shape[1],), dtype='int32')

          # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertmodel

          output_bert = self.DistilBert(
                                    input_ids=input_ids, 
                                    attention_mask=attention_mask
                                    )

          """
          last_hidden_state, 1 tensor
          hidden_states, 3 tensores (n_layers = 2 + entrada)
          attention = None (en principio)
          """
          output_bert = output_bert['hidden_states']
          """
          (batch_size, longitud_secuencia, profundidad de las capas FF intermedias)
          (batch_size, max_position_embeddings, dim=32)
          """

          # Probar tambien sumando
          output_bert = Concatenate()(output_bert)
          #dense = MaxPooling1D(pool_size=1)(output_bert)
          dense = Flatten()(output_bert)
          
          """dense2 = Reshape( (-1,1) )(word_context)
          dense2 = MaxPooling1D(pool_size=2)(dense2)
          dense2 = Flatten()(dense2)"""
          dense2 = Dense(128)(word_context)

          dense = Concatenate()([dense, dense2])

          dense = Dense(64)(dense)
          dense = BatchNormalization()(dense)
          dense = LeakyReLU()(dense)
          dense = Dropout(0.4)(dense)
          dense = Dense(32)(dense)
          dense = BatchNormalization()(dense)
          dense = relu(dense)
          dense = Dropout(0.2)(dense)

          dense = Dense(1)(dense)
          dense = BatchNormalization()(dense)
          dense = sigmoid(dense)
          output = Lambda(self.custom_activation, 
                          output_shape=(None,1))(dense)

          ### Output
          model = Model([input_ids, attention_mask, word_context], output)

          ### Cross Entropy y Optimizador Adam
          model.compile(optimizer="SGD", loss="mse", metrics=['mae', tf.keras.metrics.RootMeanSquaredError()])
          model.summary()

          self.DistilBert_model = model

    def train_model(self, x_train, y_train, x_val, y_val):
        """
        Método que entrena el modelo.

        Args:

        * x_train: matriz de diseño
        * y_train: target codificada
        """

        """
        tokens + attention_mask
        """
        print(x_train.shape)
        train_word_context = c_vec.fit_transform(x_train)
        val_word_context = c_vec.transform(x_val)

        self.DistilBertNN()
        
        train_inputs = self.get_model_inputs(cfg, x_train.to_list())
        val_inputs = self.get_model_inputs(cfg, x_val.to_list())

        # obtener tensores correspondientes
        train_blabels_t = tf.convert_to_tensor(y_train, dtype='int32')
        val_blabels_t = tf.convert_to_tensor(y_val, dtype='int32')

        # Instancio el méétodo earlyStopping para no perseverar en el entrenamiento si
        # se obtiene una mejora tras una paciencia de 8 iteraciones
        """earlyStopping = EarlyStopping(monitor='val_accuracy', patience=8, verbose=0, mode='max')"""

        # Creo un guardado del mejor modelo visto hasta el fin del entrenamiento monitorizando
        # el val_accuracy
        """checkpoint = ModelCheckpoint("best_model.h5", monitor="val_accuracy", save_best_only=True)"""

        # Ajusto el modelo con 50 epocas controladas por earlyStopiing y batch size de 50
        self.DistilBert_model.fit([train_inputs['input_ids'], train_inputs['attention_mask'], train_word_context.toarray()],  train_blabels_t, 
                  batch_size = 32, epochs = 10,
                  verbose = 1,
                  # callbacks=[earlyStopping, checkpoint],
                  validation_data=([val_inputs['input_ids'], val_inputs['attention_mask'], val_word_context.toarray()], val_blabels_t))
        #plt.hist(self.DistilBert_model.predict([train_inputs['input_ids'], train_inputs['attention_mask']]))

In [20]:
instancia = CustomDistilBERTModel(cfg, config_bert)
#instancia.DistilBertNN()

Some layers from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing TFDistilBertModel: ['bert', 'mlm___cls']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['distilbert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
instancia.train_model(x_train, y_train, x_val, y_val)

(7865,)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model (TFDistilB TFBaseModelOutput(la 1017408     input_1[0][0]                    
                                                                 input_2[0][0]                    
______________________________________________

...

Epoch 1/10


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


244/246 [============================>.] - ETA: 0s - loss: 0.7890 - mae: 0.6983 - root_mean_squared_error: 0.8883

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


246/246 [==============================] - 5s 19ms/step - loss: 0.7867 - mae: 0.6975 - root_mean_squared_error: 0.8869 - val_loss: 0.4263 - val_mae: 0.5568 - val_root_mean_squared_error: 0.6529
Epoch 2/10
246/246 [==============================] - 4s 16ms/step - loss: 0.4494 - mae: 0.5810 - root_mean_squared_error: 0.6704 - val_loss: 0.4202 - val_mae: 0.5682 - val_root_mean_squared_error: 0.6482
Epoch 3/10
246/246 [==============================] - 4s 16ms/step - loss: 0.4478 - mae: 0.5848 - root_mean_squared_error: 0.6691 - val_loss: 0.4200 - val_mae: 0.5697 - val_root_mean_squared_error: 0.6480
Epoch 4/10
246/246 [==============================] - 4s 16ms/step - loss: 0.4476 - mae: 0.5860 - root_mean_squared_error: 0.6690 - val_loss: 0.4196 - val_mae: 0.5683 - val_root_mean_squared_error: 0.6478
Epoch 5/10
246/246 [==============================] - 4s 16ms/step - loss: 0.4481 - mae: 0.5854 - root_mean_squared_error: 0.6694 - val_loss: 0.4198 - val_mae: 0.5699 - val_root_mean_squared_

---

---

---